In [ ]:
!pip install rouge
!pip install bert_score
!pip install pyemd
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import nltk
import pandas as pd
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

In [ ]:
from bert_score import score

In [ ]:
# Load CSV
df = pd.read_csv('/content/cleaned_qwenFTRAG.csv')  # Replace with your actual file name

In [ ]:
df.rename(columns={'responses': 'response'}, inplace=True)

In [ ]:
from nltk.translate.gleu_score import corpus_gleu, sentence_gleu
from nltk import word_tokenize
from collections import Counter
from sklearn.metrics import f1_score
rouge = Rouge()


def compute_gleu(reference, hypothesis):
    # Tokenize reference and hypothesis
    ref_tokens = [word_tokenize(reference.lower())]
    hyp_tokens = word_tokenize(hypothesis.lower())

    # Compute sentence-level GLEU (NLTK's implementation)
    gleu = sentence_gleu(ref_tokens, hyp_tokens)
    return gleu

def token_metrics(reference, hypothesis):
    # Tokenize and lowercase
    ref_tokens = word_tokenize(reference.lower())
    hyp_tokens = word_tokenize(hypothesis.lower())

    # Count overlaps
    ref_counts = Counter(ref_tokens)
    hyp_counts = Counter(hyp_tokens)
    common_tokens = ref_counts & hyp_counts
    tp = sum(common_tokens.values())

    # Precision/Recall/F1
    precision = tp / len(hyp_tokens) if len(hyp_tokens) > 0 else 0
    recall = tp / len(ref_tokens) if len(ref_tokens) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

def compute_scores(reference, hypothesis):
    # BLEU Score
    bleu = sentence_bleu([reference.split()], hypothesis.split(),
                         smoothing_function=SmoothingFunction().method1)

    # GLEU Score
    gleu = compute_gleu(reference, hypothesis)

    # ROUGE Scores
    rouge_scores = rouge.get_scores(hypothesis, reference)[0]
    rouge_1 = rouge_scores['rouge-1']['f']
    rouge_2 = rouge_scores['rouge-2']['f']
    rouge_l = rouge_scores['rouge-l']['f']

    # METEOR Score
    meteor = meteor_score([reference.split()], hypothesis.split())

    # Token-level metrics (Precision, Recall, F1)
    precision, recall, token_f1 = token_metrics(reference, hypothesis)

    return bleu, gleu, rouge_1, rouge_2, rouge_l, meteor, precision, recall, token_f1

# Apply evaluation to each row
df[['BLEU', 'GLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'METEOR', 'Precision', 'Recall', 'Token_F1']] = df.apply(
    lambda row: compute_scores(row['output'], row['response']), axis=1, result_type='expand'
)

# Compute BERTScore
P, R, F1 = score(df['response'].tolist(), df['output'].tolist(), lang='en', model_type='roberta-large')
df['BERTScore_P'] = P.tolist()
df['BERTScore_R'] = R.tolist()
df['BERTScore_F1'] = F1.tolist()


# Display results
print(df[['input', 'BLEU', 'GLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'METEOR', 'Precision', 'Recall', 'Token_F1', 'BERTScore_F1']].head())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                                               input      BLEU      GLEU  \
0  What is (are) ornithine transcarbamylase defic...  0.079868  0.153753   
1                    What is (are) globozoospermia ?  0.000051  0.076923   
2                 What is (are) hypochondrogenesis ?  0.055638  0.265509   
3               What is (are) Macular Degeneration ?  0.001279  0.057545   
4  What is (are) familial hemophagocytic lymphohi...  0.020469  0.157068   

    ROUGE-1   ROUGE-2   ROUGE-L    METEOR  Precision    Recall  Token_F1  \
0  0.295964  0.084967  0.278027  0.276829   0.408654  0.512048  0.454545   
1  0.262295  0.160920  0.262295  0.079804   1.000000  0.090909  0.166667   
2  0.445860  0.415459  0.445860  0.212430   1.000000  0.270936  0.426357   
3  0.179310  0.041451  0.165517  0.087458   0.634615  0.167513  0.265060   
4  0.330827  0.089239  0.308271  0.215445   0.589623  0.434028  0.500000   

   BERTScore_F1  
0      0.880492  
1      0.885974  
2      0.915646  
3      0.85627



---



In [ ]:
# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute SBERT cosine similarity
sbert_similarities = []
for ref, hyp in zip(df['output'], df['response']):
    ref_emb = sbert_model.encode(ref, convert_to_tensor=True)
    hyp_emb = sbert_model.encode(hyp, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(ref_emb, hyp_emb).item()
    sbert_similarities.append(similarity)

df['SBERT_Cosine_Similarity'] = sbert_similarities

In [ ]:
df

,instruction,input,output,response,BLEU,GLEU,ROUGE-1,ROUGE-2,ROUGE-L,METEOR,Precision,Recall,Token_F1,BERTScore_P,BERTScore_R,BERTScore_F1,SBERT_Cosine_Similarity
0,Answer this question truthfully,What is (are) ornithine transcarbamylase defic...,Ornithine transcarbamylase deficiency is an in...,*Ornithine transcarbamylase (OTC) deficiency i...,0.044970,0.146532,0.362869,0.103976,0.345992,0.275802,0.413333,0.560241,0.475703,0.860387,0.886359,0.873180,0.874685
1,Answer this question truthfully,What is (are) globozoospermia ?,Globozoospermia is a condition that affects on...,It is a condition characterized by the format...,0.023120,0.115385,0.296296,0.079470,0.296296,0.218795,0.428571,0.363636,0.393443,0.850091,0.851078,0.850584,0.621321
2,Answer this question truthfully,What is (are) hypochondrogenesis ?,"Hypochondrogenesis is a rare, severe disorder ...","Hypochondrogenesis is a rare, severe disorder ...",0.319619,0.413151,0.531250,0.449612,0.531250,0.406914,0.824000,0.507389,0.628049,0.949064,0.887248,0.917116,0.821477
3,Answer this question truthfully,What is (are) Macular Degeneration ?,"Macular degeneration, or age-related macular d...",Age-related macular degeneration is a leading ...,0.016122,0.084399,0.240602,0.127660,0.240602,0.145988,0.696429,0.197970,0.308300,0.900356,0.830070,0.863785,0.774451
4,Answer this question truthfully,What is (are) familial hemophagocytic lymphohi...,Familial hemophagocytic lymphohistiocytosis is...,The condition begins in fetal or early childho...,0.004611,0.079407,0.200000,0.016349,0.200000,0.131416,0.417112,0.270833,0.328421,0.826558,0.841565,0.833994,0.602024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Answer this question truthfully,What is (are) hereditary hemorrhagic telangiec...,Hereditary hemorrhagic telangiectasia is a dis...,Hereditary (passed down from parents to child...,0.002730,0.063329,0.225681,0.058680,0.210117,0.100396,0.581395,0.180288,0.275229,0.842047,0.820923,0.831351,0.778471
71,Answer this question truthfully,What is (are) Lamellar ichthyosis ?,Lamellar ichthyosis is a rare genetic conditio...,Lamellar ichthyosis is a rare genetic conditio...,0.448972,0.453258,0.568306,0.475836,0.546448,0.498887,0.601124,0.677215,0.636905,0.922028,0.918989,0.920506,0.923395
72,Answer this question truthfully,What is (are) Periventricular Leukomalacia ?,Periventricular leukomalacia (PVL) is characte...,Periventricular Leukomalacia (PVL) is a disord...,0.004047,0.099424,0.302521,0.157576,0.285714,0.095919,0.843750,0.154286,0.260870,0.943218,0.851184,0.894841,0.892288
73,Answer this question truthfully,What is (are) Dentinogenesis imperfecta ?,Dentinogenesis imperfecta is a condition that ...,Dentinogenesis imperfecta is an inherited cond...,0.034652,0.130108,0.316742,0.084034,0.316742,0.232246,0.401709,0.479592,0.437209,0.857649,0.853466,0.855553,0.771353


In [ ]:
df.to_csv('evaluation_results_qwenFTRAG.csv', index=False)

In [ ]:
print(df['BLEU'].mean())
print(df['GLEU'].mean())
print(df['ROUGE-1'].mean())
print(df['ROUGE-2'].mean())
print(df['ROUGE-L'].mean())
print(df['METEOR'].mean())
print(df['Precision'].mean())
print(df['Recall'].mean())
print(df['Token_F1'].mean())
print(df['BERTScore_P'].mean())
print(df['BERTScore_R'].mean())
print(df['BERTScore_F1'].mean())
print(df['SBERT_Cosine_Similarity'].mean())

0.04163774251428854
0.12995138711470114
0.3022744132995979
0.16616246147956562
0.28967792378235346
0.16001128819278101
0.715645251818578
0.23153929974401427
0.31632186109674765
0.9111330183347066
0.8377828057607015
0.8723959986368816
0.7873402492205301
